### 地域別訪日客数推移

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

def scrape_visitor_stats():
    url = "https://www.tourism.jp/tourism-database/stats/inbound/#monthly"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'ja,en-US;q=0.9,en;q=0.8'
    }
    
    try:
        time.sleep(1)
        
        session = requests.Session()
        response = session.get(url, headers=headers)
        response.encoding = 'utf-8'
        
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table')
        
        visitor_data = []
        
        # テーブル内のすべてのテキストを取得し処理
        all_text = table.get_text()
        # 行ごとに分割
        lines = [line.strip() for line in all_text.split('\n') if line.strip()]
        
        # 国名と人数のペアを探す
        current_country = None
        for text in lines:
            # 人数を含む行を処理
            if '人' in text and text.replace(',', '').replace('人', '').strip().isdigit():
                if current_country:  # 直前の国名があれば、ペアとして追加
                    visitors = int(text.replace(',', '').replace('人', '').strip())
                    visitor_data.append({
                        '国/地域': current_country,
                        '訪問者数': visitors
                    })
                current_country = None
            else:
                # 人数を含まない行は国名として扱う
                current_country = text.strip()

        df = pd.DataFrame(visitor_data)
        if not df.empty:
            print("\n国・地域別訪日客数:")
            print(df)
            return df
    
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")
        return pd.DataFrame()

if __name__ == "__main__":
    visitor_stats = scrape_visitor_stats()


国・地域別訪日客数:
       国/地域    訪問者数
0        韓国  749500
1        中国  546300
2        台湾  488400
3        香港  227100
4        タイ  118000
5    シンガポール   95800
6     マレーシア   62000
7    インドネシア   48800
8     フィリピン   87100
9      ベトナム   50200
10      インド   23500
11       英国   37000
12     フランス   31100
13      ドイツ   25200
14     イタリア   20800
15      ロシア   13900
16     スペイン   18800
17      USA  247500
18      カナダ   47300
19  オーストラリア   80300


### 地域別日本人出国者数推移

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

def scrape_outbound_stats():
    url = "https://www.tourism.jp/tourism-database/stats/outbound/#monthly"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'ja,en-US;q=0.9,en;q=0.8'
    }
    
    try:
        time.sleep(1)
        
        session = requests.Session()
        response = session.get(url, headers=headers)
        response.encoding = 'utf-8'
        
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table')
        
        outbound_data = []
        
        all_text = table.get_text()
        lines = [line.strip() for line in all_text.split('\n') if line.strip()]
        
        current_region = None
        for text in lines:
            if '人' in text and text.replace(',', '').replace('人', '').strip().isdigit():
                if current_region:
                    visitors = int(text.replace(',', '').replace('人', '').strip())
                    outbound_data.append({
                        '国/地域': current_region,
                        '出国者数': visitors
                    })
                current_region = None
            else:
                current_region = text.strip()

        df = pd.DataFrame(outbound_data)
        if not df.empty:
            print("\n国・地域別日本人出国者数:")
            print(df)
            return df
    
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")
        return pd.DataFrame()

if __name__ == "__main__":
    outbound_stats = scrape_outbound_stats()


国・地域別日本人出国者数:
       国/地域    出国者数
0        韓国  341198
1        香港   54389
2       マカオ   11719
3        タイ  109639
4     マレーシア   23498
5    シンガポール   44330
6     フィリピン   30717
7      ベトナム   71014
8      アメリカ  141389
9  ハワイ/アメリカ   60683


### 実際にスクレイピングからDBに格納までをまとめる (片方しかデータがない国は欠損地として削除)

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import sqlite3
from datetime import datetime

# データベース関連の関数
def create_database():
   """データベースとテーブルの作成"""
   conn = sqlite3.connect('tourism_stats.db')
   cursor = conn.cursor()
   
   cursor.execute('''
   CREATE TABLE IF NOT EXISTS tourism_statistics (
       id INTEGER PRIMARY KEY AUTOINCREMENT,
       country TEXT NOT NULL,
       inbound_visitors INTEGER,
       outbound_visitors INTEGER,
       date_updated DATE
   )
   ''')
   
   conn.commit()
   conn.close()

def standardize_country_name(country):
   """国名を標準化する"""
   country_mapping = {
       'USA': 'アメリカ',
       'United States': 'アメリカ',
       '米国': 'アメリカ',
       # 必要に応じて他の国の表記揺れも追加
   }
   return country_mapping.get(country, country)

def store_data_in_db(inbound_df, outbound_df):
   """スクレイピングしたデータをDBに保存"""
   conn = sqlite3.connect('tourism_stats.db')
   
   # 国名の標準化
   inbound_df['国/地域'] = inbound_df['国/地域'].apply(standardize_country_name)
   outbound_df['国/地域'] = outbound_df['国/地域'].apply(standardize_country_name)
   
   # 同じ国のデータを集計
   inbound_df = inbound_df.groupby('国/地域')['訪日者数'].sum().reset_index()
   outbound_df = outbound_df.groupby('国/地域')['出国者数'].sum().reset_index()
   
   # 両方のデータが存在する国のみを抽出（内部結合）
   combined_df = pd.merge(inbound_df, outbound_df, on='国/地域', how='inner')
   combined_df['date_updated'] = datetime.now().date()
   
   # カラム名の変更
   combined_df = combined_df.rename(columns={
       '国/地域': 'country',
       '訪日者数': 'inbound_visitors',
       '出国者数': 'outbound_visitors'
   })
   
   combined_df.to_sql('tourism_statistics', conn, if_exists='replace', index=False)
   
   conn.commit()
   conn.close()
   
   # データの確認用出力
   print(f"\n保存されたデータ（両方のデータが存在する国のみ）:")
   print(f"データ数: {len(combined_df)}件")
   print(combined_df)

# スクレイピング関連の関数
def scrape_visitor_stats():
   """訪日観光客データのスクレイピング"""
   url = "https://www.tourism.jp/tourism-database/stats/inbound/#monthly"
   
   headers = {
       'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
       'Accept-Language': 'ja,en-US;q=0.9,en;q=0.8'
   }
   
   try:
       time.sleep(1)
       session = requests.Session()
       response = session.get(url, headers=headers)
       response.encoding = 'utf-8'
       
       soup = BeautifulSoup(response.text, 'html.parser')
       table = soup.find('table')
       
       visitor_data = []
       all_text = table.get_text()
       lines = [line.strip() for line in all_text.split('\n') if line.strip()]
       
       current_country = None
       for text in lines:
           if '人' in text and text.replace(',', '').replace('人', '').strip().isdigit():
               if current_country:
                   visitors = int(text.replace(',', '').replace('人', '').strip())
                   visitor_data.append({
                       '国/地域': current_country,
                       '訪日者数': visitors
                   })
               current_country = None
           else:
               current_country = text.strip()

       return pd.DataFrame(visitor_data)
   
   except Exception as e:
       print(f"訪日データの取得でエラーが発生しました: {str(e)}")
       return pd.DataFrame()

def scrape_outbound_stats():
   """日本人出国者データのスクレイピング"""
   url = "https://www.tourism.jp/tourism-database/stats/outbound/#monthly"
   
   headers = {
       'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
       'Accept-Language': 'ja,en-US;q=0.9,en;q=0.8'
   }
   
   try:
       time.sleep(1)
       session = requests.Session()
       response = session.get(url, headers=headers)
       response.encoding = 'utf-8'
       
       soup = BeautifulSoup(response.text, 'html.parser')
       table = soup.find('table')
       
       outbound_data = []
       all_text = table.get_text()
       lines = [line.strip() for line in all_text.split('\n') if line.strip()]
       
       current_region = None
       for text in lines:
           if '人' in text and text.replace(',', '').replace('人', '').strip().isdigit():
               if current_region:
                   visitors = int(text.replace(',', '').replace('人', '').strip())
                   outbound_data.append({
                       '国/地域': current_region,
                       '出国者数': visitors
                   })
               current_region = None
           else:
               current_region = text.strip()

       return pd.DataFrame(outbound_data)
   
   except Exception as e:
       print(f"出国データの取得でエラーが発生しました: {str(e)}")
       return pd.DataFrame()

def main():
   """メイン実行関数"""
   # データベースの作成
   create_database()
   
   # データの取得とDB保存
   print("データの取得を開始します...")
   inbound_df = scrape_visitor_stats()
   outbound_df = scrape_outbound_stats()
   
   if not inbound_df.empty and not outbound_df.empty:
       print("データベースに保存します...")
       store_data_in_db(inbound_df, outbound_df)
       print("データベースの更新が完了しました")
   else:
       print("データの取得に失敗しました")

if __name__ == "__main__":
   main()

データの取得を開始します...
データベースに保存します...

保存されたデータ（両方のデータが存在する国のみ）:
データ数: 8件
  country  inbound_visitors  outbound_visitors date_updated
0    アメリカ            247500             141389   2025-01-23
1  シンガポール             95800              44330   2025-01-23
2      タイ            118000             109639   2025-01-23
3   フィリピン             87100              30717   2025-01-23
4    ベトナム             50200              71014   2025-01-23
5   マレーシア             62000              23498   2025-01-23
6      韓国            749500             341198   2025-01-23
7      香港            227100              54389   2025-01-23
データベースの更新が完了しました


### DBから情報を取得し計算

In [41]:
import sqlite3
import pandas as pd
from scipy import stats

def get_continent(country):
    """国を大陸（地域）に分類"""
    continent_mapping = {
        'アジア': ['中国','香港', '韓国', '台湾', 'タイ', 'フィリピン', 'マレーシア', 'シンガポール', 
                 'インドネシア', 'ベトナム', 'インド'],
        '欧州': ['イギリス', 'フランス', 'ドイツ', 'イタリア', 'スペイン', 'ロシア', 
               'オランダ', 'スイス', 'スウェーデン'],
        '北米': ['アメリカ', 'カナダ'],
        'オセアニア': ['オーストラリア', 'ニュージーランド'],
        'その他': []  # 上記以外の国
    }
    
    for continent, countries in continent_mapping.items():
        if country in countries:
            return continent
    return 'その他'

def analyze_correlation_by_continent():
    """大陸別の相関関係分析"""
    try:
        # データベースに接続
        conn = sqlite3.connect('tourism_stats.db')
        
        # データの取得
        query = """
        SELECT country, inbound_visitors, outbound_visitors
        FROM tourism_statistics
        ORDER BY country
        """
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        # 大陸情報を追加
        df['continent'] = df['country'].apply(get_continent)
        
        # 全体の相関
        overall_corr = stats.pearsonr(df['inbound_visitors'], df['outbound_visitors'])
        print("\n全体の相関関係:")
        print(f"ピアソン相関係数: {overall_corr[0]:.4f}")
        print(f"p値: {overall_corr[1]:.4f}")
        print("---" * 20)
        
        # 大陸ごとの分析
        for continent in sorted(df['continent'].unique()):
            continent_df = df[df['continent'] == continent]
            
            print(f"\n{continent}の分析:")
            print(f"国数: {len(continent_df)}件")
            
            if len(continent_df) > 1:  # 相関係数を計算するには2つ以上のデータが必要
                corr = stats.pearsonr(
                    continent_df['inbound_visitors'], 
                    continent_df['outbound_visitors']
                )
                print(f"相関係数: {corr[0]:.4f}")
                print(f"p値: {corr[1]:.4f}")
            
            print("\n国別データ:")
            for _, row in continent_df.iterrows():
                print(f"{row['country']:<15} 訪日: {row['inbound_visitors']:>10,} 出国: {row['outbound_visitors']:>10,}")
            
            # 基本統計量
            print("\n基本統計量:")
            print(continent_df[['inbound_visitors', 'outbound_visitors']].describe())
            print("---" * 20)
            
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")

if __name__ == "__main__":
    analyze_correlation_by_continent()


全体の相関関係:
ピアソン相関係数: 0.9474
p値: 0.0004
------------------------------------------------------------

アジアの分析:
国数: 7件
相関係数: 0.9497
p値: 0.0011

国別データ:
シンガポール          訪日:     95,800 出国:     44,330
タイ              訪日:    118,000 出国:    109,639
フィリピン           訪日:     87,100 出国:     30,717
ベトナム            訪日:     50,200 出国:     71,014
マレーシア           訪日:     62,000 出国:     23,498
韓国              訪日:    749,500 出国:    341,198
香港              訪日:    227,100 出国:     54,389

基本統計量:
       inbound_visitors  outbound_visitors
count          7.000000           7.000000
mean      198528.571429       96397.857143
std       249827.138333      111692.246823
min        50200.000000       23498.000000
25%        74550.000000       37523.500000
50%        95800.000000       54389.000000
75%       172550.000000       90326.500000
max       749500.000000      341198.000000
------------------------------------------------------------

北米の分析:
国数: 1件

国別データ:
アメリカ            訪日:    247,500 出国:    141,389

基本統計量